<center> 
     <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300"> 
</center>

# Assignment: Notebook for Peer Assignment
Estimated time needed: 45 minutes


# Assignment Scenario

Congratulations! You have just been hired by a US Venture Capital firm as a data analyst.

The company is considering foreign grain markets to help meet its supply chain requirements for its recent investments in the microbrewery and microdistillery industry, which is involved with the production and distribution of craft beers and spirits.

Your first task is to provide a high level analysis of crop production in Canada. Your stakeholders want to understand the current and historical performance of certain crop types in terms of supply and price volatility. For now they are mainly interested in a macro-view of Canada's crop farming industry, and how it relates to the relative value of the Canadian and US dollars.


# Introduction

Using this R notebook you will:

1.  Understand four datasets 
2.  Load the datasets into four separate tables in a Db2 database
3.  Execute SQL queries unsing the RODBC R package to answer assignment questions 

You have already encountered two of these datasets in the previous practice lab. You will be able to reuse much of the work you did there to prepare your database tables for executing SQL queries.


# Understand the datasets

To complete the assignment problems in this notebook you will be using subsetted snapshots of two datasets from Statistics Canada, and one from the Bank of Canada. The links to the prepared datasets are provided in the next section; the interested student can explore the landing pages for the source datasets as follows:

1.  <a href="https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMRP0203ENSkillsNetwork890-2022-01-01&pid=3210035901">Canadian Principal Crops (Data & Metadata)</a>
2.  <a href="https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMRP0203ENSkillsNetwork890-2022-01-01&pid=3210007701">Farm product prices (Data & Metadata)</a>
3.  <a href="https://www.bankofcanada.ca/rates/exchange/daily-exchange-rates?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMRP0203ENSkillsNetwork890-2022-01-01">Bank of Canada daily average exchange rates</a>


### 1. Canadian Principal Crops Data *

This dataset contains agricultural production measures for the principle crops grown in Canada, including a breakdown by province and teritory, for each year from 1908 to 2020.

For this assignment you will use a preprocessed snapshot of this dataset (see below).

A detailed description of this dataset can be obtained from the StatsCan Data Portal at:
https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3210035901  
Detailed information is included in the metadata file and as header text in the data file, which can be downloaded - look for the 'download options' link.  

### 2. Farm product prices

This dataset contains monthly average farm product prices for Canadian crops and livestock by province and teritory, from 1980 to 2020 (or 'last year', whichever is greatest).

For this assignment you will use a preprocessed snapshot of this dataset (see below).

A description of this dataset can be obtained from the StatsCan Data Portal at:
https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3210007701 
The information is included in the metadata file, which can be downloaded - look for the 'download options' link.  

### 3. Bank of Canada daily average exchange rates *

This dataset contains the daily average exchange rates for multiple foreign currencies. Exchange rates are expressed as 1 unit of the foreign currency converted into Canadian dollars. It includes only the latest four years of data, and the rates are published once each business day by 16:30 ET.

For this assignment you will use a snapshot of this dataset with only the USD-CAD exchange rates included (see next section). We have also prepared a monthly averaged version which you will be using below.

A brief description of this dataset and the original dataset can be obtained from the Bank of Canada Data Portal at:
https://www.bankofcanada.ca/rates/exchange/daily-exchange-rates/

( * these datasets are the same as the ones you used in the practice lab)


### Dataset URLs

  1.  Annual Crop Data: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Annual_Crop_Data.csv 

  2.  Farm product prices: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_Farm_Prices.csv
  
  3.  Daily FX Data: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Daily_FX.csv
  
  4.  Monthly FX Data: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_FX.csv
  

<span style="color:red">**IMPORTANT:**</span> You will be loading these datasets directly into R data frames from these URLs instead of from the StatsCan and Bank of Canada portals. The versions provided at these URLs are simplified and subsetted versions of the original datasets.


#### Now let's load these datasets into four separate Db2 tables.
Let's first load the RODBC package:


In [5]:
library(RODBC)

## Problem 1
#### Create tables
Establish a connection to the Db2 database, and create the following four tables using the RODBC package in R. 
Use the separate cells provided below to create each of your tables.

1.  **CROP_DATA**
2.  **FARM_PRICES**
3.  **DAILY_FX**
4.  **MONTHLY_FX**  

The previous practice lab will help you accomplish this.


### Solution 1


In [7]:
# Establish database connection
dsn_driver <- "{IBM DB2 ODBC Driver}"
dsn_database <- "bludb"            
dsn_hostname <- "54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud" 
dsn_port <- "32733"    
dsn_protocol <- "TCPIP"            
dsn_uid <- "hfg30691"        
dsn_pwd <- "99w1ORi8rwqGsNAw"        
dsn_security <- "ssl"

conn_path <- paste("DRIVER=",dsn_driver,
                  ";DATABASE=",dsn_database,
                  ";HOSTNAME=",dsn_hostname,
                  ";PORT=",dsn_port,
                  ";PROTOCOL=",dsn_protocol,
                  ";UID=",dsn_uid,
                  ";PWD=",dsn_pwd,
                  ";SECURITY=",dsn_security,        
                    sep="")
conn <- odbcDriverConnect(conn_path)

   
sql.info <- sqlTypeInfo(conn)
conn.info <- odbcGetInfo(conn)
conn.info["DBMS_Name"]
conn.info["DBMS_Ver"]
conn.info["Driver_ODBC_Ver"]

tables <- c("CROP_DATA", "FARM_PRICES", "DAILY_FX", "MONTHLY_FX") 

    for (table in tables) {
        # Drop tables if they already exist
        out <- sqlTables(conn, tableType = "TABLE",
                          tableName = table)
        if (nrow(out)>0) {
            err <- sqlDrop(conn, table,
                            errors=FALSE)  
            if (err==-1) {
                cat("An error has occurred.\n")
                err.msg <- odbcGetErrMsg(conn)
                for (error in err.msg) { 
                    cat(error,"\n")
                }
            } 
            else {
                cat ("Table: ",table," was dropped\n")
            }
        }
        else {
              cat ("Table: ", table," does not exist\n")
        }
    }

DBMS_Name 
"DB2/LINUXX8664"

DBMS_Ver 
"11.05.0800"

Driver_ODBC_Ver 
        "03.51"

Table:  CROP_DATA  does not exist
Table:  FARM_PRICES  does not exist
Table:  DAILY_FX  does not exist
Table:  MONTHLY_FX  does not exist


In [8]:
# CROP_DATA:
df1 <- sqlQuery(conn, 
                    "CREATE TABLE CROP_DATA (
                                      CD_ID INTEGER NOT NULL,
                                      YEAR DATE NOT NULL,
                                      CROP_TYPE VARCHAR(20) NOT NULL,
                                      GEO VARCHAR(20) NOT NULL, 
                                      SEEDED_AREA INTEGER NOT NULL,
                                      HARVESTED_AREA INTEGER NOT NULL,
                                      PRODUCTION INTEGER NOT NULL,
                                      AVG_YIELD INTEGER NOT NULL,
                                      PRIMARY KEY (CD_ID)
                                      )", 
                    errors=FALSE
                    )

    if (df1 == -1){
        cat ("An error has occurred.\n")
        msg <- odbcGetErrMsg(conn)
        print (msg)
    } else {
        cat ("Table was created successfully.\n")
    }

Table was created successfully.


In [9]:
# FARM_PRICES:
df2 <- sqlQuery(conn, 
                    "CREATE TABLE FARM_PRICES (
                                      CD_ID INTEGER NOT NULL,
                                      DATE DATE NOT NULL,
                                      CROP_TYPE VARCHAR(255) NOT NULL,
                                      GEO VARCHAR(255) NOT NULL, 
                                      PRICE_PRERMT INTEGER NOT NULL,
                                      PRIMARY KEY (CD_ID)
                                      )", 
                    errors=FALSE
                    )

    if (df2 == -1){
        cat ("An error has occurred.\n")
        msg <- odbcGetErrMsg(conn)
        print (msg)
    } else {
        cat ("Table was created successfully.\n")
    }

Table was created successfully.


In [10]:
# DAILY_FX:
df3 <- sqlQuery(conn, "CREATE TABLE DAILY_FX (
                                DFX_ID INTEGER NOT NULL,
                                DATE DATE NOT NULL, 
                                FXUSDCAD FLOAT(6),
                                PRIMARY KEY (DFX_ID)
                                )",
                    errors=FALSE
                    )

    if (df3 == -1){
        cat ("An error has occurred.\n")
        msg <- odbcGetErrMsg(conn)
        print (msg)
    } else {
        cat ("Table was created successfully.\n")
    }

Table was created successfully.


In [11]:
# MONTHLY_FX:
df4 <- sqlQuery(conn, "CREATE TABLE MONTHLY_FX (
                                MFX_ID INTEGER NOT NULL,
                                DATE DATE NOT NULL, 
                                FXUSDCAD FLOAT(6),
                                PRIMARY KEY (MFX_ID)
                                )",
                    errors=FALSE
                    )

    if (df4 == -1){
        cat ("An error has occurred.\n")
        msg <- odbcGetErrMsg(conn)
        print (msg)
    } else {
        cat ("Table was created successfully.\n")
    }

Table was created successfully.


## Problem 2
#### Read Datasets and Load Tables
Read the datasets into R dataframes using the urls provided above. Then load your tables.


###  Solution 2


In [12]:
crop_df <-read.csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Annual_Crop_Data.csv', colClasses=c(year="character"))
farm_df <-read.csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_Farm_Prices.csv', colClasses=c(MONTHLY="character"))
daily_df <-read.csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Daily_FX.csv', colClasses=c(date="character"))
monthly_df <-read.csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_FX.csv', colClasses=c(MONTHLY="character"))

head(crop_df)
head(farm_df)
head(daily_df)
head(monthly_df)

sqlSave(conn, crop_df, "CROP_DATA", append=TRUE, fast=FALSE, rownames=FALSE, colnames=FALSE, verbose=FALSE)
sqlSave(conn, farm_df, "FARM_PRICES", append=TRUE, fast=FALSE, rownames=FALSE, colnames=FALSE, verbose=FALSE)
sqlSave(conn, daily_df, "DAILY_FX", append=TRUE, fast=FALSE, rownames=FALSE, colnames=FALSE, verbose=FALSE)
sqlSave(conn, monthly_df, "MONTHLY_FX", append=TRUE, fast=FALSE, rownames=FALSE, colnames=FALSE, verbose=FALSE)



Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“not all columns named in 'colClasses' exist”Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“not all columns named in 'colClasses' exist”Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“not all columns named in 'colClasses' exist”Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“not all columns named in 'colClasses' exist”

,CD_ID,YEAR,CROP_TYPE,GEO,SEEDED_AREA,HARVESTED_AREA,PRODUCTION,AVG_YIELD
,<int>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>
1,0,1965-12-31,Barley,Alberta,1372000,1372000,2504000,1825
2,1,1965-12-31,Barley,Canada,2476800,2476800,4752900,1920
3,2,1965-12-31,Barley,Saskatchewan,708000,708000,1415000,2000
4,3,1965-12-31,Canola,Alberta,297400,297400,215500,725
5,4,1965-12-31,Canola,Canada,580700,580700,512600,885
6,5,1965-12-31,Canola,Saskatchewan,224600,224600,242700,1080


,CD_ID,DATE,CROP_TYPE,GEO,PRICE_PRERMT
,<int>,<fct>,<fct>,<fct>,<dbl>
1,0,1985-01-01,Barley,Alberta,127.39
2,1,1985-01-01,Barley,Saskatchewan,121.38
3,2,1985-01-01,Canola,Alberta,342.00
4,3,1985-01-01,Canola,Saskatchewan,339.82
5,4,1985-01-01,Rye,Alberta,100.77
6,5,1985-01-01,Rye,Saskatchewan,109.75


,DFX_ID,DATE,FXUSDCAD
,<int>,<fct>,<dbl>
1,0,2017-01-03,1.3435
2,1,2017-01-04,1.3315
3,2,2017-01-05,1.3244
4,3,2017-01-06,1.3214
5,4,2017-01-09,1.3240
6,5,2017-01-10,1.3213


,DFX_ID,DATE,FXUSDCAD
,<int>,<fct>,<dbl>
1,0,2017-01-01,1.319276
2,1,2017-02-01,1.310726
3,2,2017-03-01,1.338643
4,3,2017-04-01,1.344021
5,4,2017-05-01,1.360705
6,5,2017-06-01,1.329805


## Now execute SQL queries using the RODBC R package to solve the assignment problems.

## Problem 3
#### How many records are in the farm prices dataset?


### Solution 3


In [13]:
query <- "SELECT COUNT(*) AS RECORD 
          FROM FARM_PRICES;"

view <- sqlQuery(conn,query)
view

,RECORD
,<int>
1,2678


## Problem 4
#### Which geographies are included in the farm prices dataset?


### Solution 4


In [14]:
query <- "SELECT DISTINCT(GEO) AS CITY
          FROM FARM_PRICES;"

view <- sqlQuery(conn,query)
view

,CITY
,<fct>
1,Alberta
2,Saskatchewan


## Problem 5
#### How many hectares of Rye were harvested in Canada in 1968?


### Solution 5


In [15]:
query <-"SELECT SUM(HARVESTED_AREA) AS TOTAL_AREA_HARVESTED
        FROM CROP_DATA WHERE GEO = 'Canada' AND CROP_TYPE = 'Rye' AND YEAR(YEAR) = 1968;"

view <- sqlQuery(conn,query)
view

,TOTAL_AREA_HARVESTED
,<int>
1,274100


## Problem 6
#### Query and display the first 6 rows of the farm prices table for Rye.


### Solution 6


In [16]:
query <- "SELECT * FROM FARM_PRICES WHERE CROP_TYPE='Rye' LIMIT 6;"
view <- sqlQuery(conn,query)
view

,CD_ID,DATE,CROP_TYPE,GEO,PRICE_PRERMT
,<int>,<date>,<fct>,<fct>,<int>
1,4,1985-01-01,Rye,Alberta,100
2,5,1985-01-01,Rye,Saskatchewan,109
3,10,1985-02-01,Rye,Alberta,95
4,11,1985-02-01,Rye,Saskatchewan,103
5,16,1985-03-01,Rye,Alberta,96
6,17,1985-03-01,Rye,Saskatchewan,106


## Problem 7
#### Which provinces grew Barley? 


### Solution 7


In [27]:
query <- "SELECT DISTINCT(CROP_TYPE) AS TYPE_CROP, GEO FROM CROP_DATA
          WHERE CROP_TYPE = 'Barley';"
    
view <- sqlQuery(conn,query)
view

,TYPE_CROP,GEO
,<fct>,<fct>
1,Barley,Alberta
2,Barley,Canada
3,Barley,Saskatchewan


## Problem 8
#### Find the first and last dates for the farm prices data.


### Solution 8


In [21]:
query <-"SELECT min(DATE) FIRST_DATE, max(DATE) LAST_DATE
    FROM FARM_PRICES;"
view <- sqlQuery(conn,query)
view

,FIRST_DATE,LAST_DATE
,<date>,<date>
1,1985-01-01,2020-12-01


## Problem 9
#### Which crops have ever reached a farm price greater than or equal to &#0036;350 per metric tonne?


### Solution 9


In [29]:
query <-"SELECT CROP_TYPE, PRICE_PRERMT FROM FARM_PRICES
         WHERE PRICE_PRERMT >=350;"
 
view <- sqlQuery(conn,query)
view

,CROP_TYPE,PRICE_PRERMT
,<fct>,<int>
1,Canola,350
2,Canola,363
3,Canola,364
4,Canola,354
5,Canola,359
6,Canola,350
7,Canola,354
8,Canola,354
9,Canola,357


## Problem 10
#### Rank the crop types harvested in Saskatchewan in the year 2000 by their average yield. Which crop performed best? 


### Solution 10


In [31]:
query <-"SELECT YEAR(YEAR) AS YEAR, GEO, AVG_YIELD 
    FROM CROP_DATA WHERE YEAR(YEAR)=2000 AND GEO='Saskatchewan'
    ORDER BY AVG_YIELD ASC;"

view <- sqlQuery(conn,query)
view

,YEAR,GEO,AVG_YIELD
,<int>,<fct>,<int>
1,2000,Saskatchewan,1400
2,2000,Saskatchewan,2100
3,2000,Saskatchewan,2200
4,2000,Saskatchewan,2800


## Problem 11
#### Rank the crops and geographies by their average yield (KG per hectare) since the year 2000. Which crop and province had the highest average yield since the year 2000? 


### Solution 11


In [24]:
query <-"SELECT CROP_TYPE AS RANK_CROP, GEO, AVG_YIELD AS AVG_YIELD_YEAR2000 
             FROM  CROP_DATA WHERE YEAR(YEAR) >=2000 ORDER BY AVG_YIELD_YEAR2000 DESC LIMIT 13;"

view <- sqlQuery(conn,query)
view

,RANK_CROP,GEO,AVG_YIELD_YEAR2000
,<fct>,<fct>,<int>
1,Barley,Alberta,4100
2,Barley,Alberta,4100
3,Barley,Alberta,3980
4,Wheat,Alberta,3900
5,Barley,Alberta,3900
6,Wheat,Alberta,3900
7,Barley,Canada,3900
8,Barley,Alberta,3890
9,Barley,Canada,3820


## Problem 12
#### Use a subquery to determine how much wheat was harvested in Canada in the most recent year of the data.


### Solution 12


In [25]:
query <-"SELECT SUM(HARVESTED_AREA) AS HARVESTED_WHEAT
        FROM CROP_DATA WHERE YEAR(YEAR)= (SELECT MAX(YEAR(YEAR)) FROM CROP_DATA
        WHERE GEO = 'Canada' AND CROP_TYPE = 'Wheat')"
view <- sqlQuery(conn,query)
view

,HARVESTED_WHEAT
,<int>
1,38897100


## Problem 13
#### Use an implicit inner join to calculate the monthly price per metric tonne of Canola grown in Saskatchewan in both Canadian and US dollars. Display the most recent 6 months of the data.


### Solution 13


In [26]:
query <- "SELECT F.DATE, F.CORP_TYPE, M.FXUSDCAD,F.PRICE_PRERMT AS PRICE_CAD, (F.PRICE_PRERMT/ M.FXUSDCAD) AS PRICE_USD 
         FROM FARM_PRICES, MONTHLY_FX 
         WHERE F.DATE=M.DATE AND F.CORP_TYPE='Canola' AND GEO='Saskatchewan' ORDER BY F.DATE DESC LIMIT 6"

view<- sqlQuery(conn, query)
view

[1] "42S22 -206 [IBM][CLI Driver][DB2/LINUXX8664] SQL0206N  \"F.DATE\" is not valid in the context where it is used.  SQLSTATE=42703\n"                                                                                                                                                                         
[2] "[RODBC] ERROR: Could not SQLExecDirect 'SELECT F.DATE, F.CORP_TYPE, M.FXUSDCAD,F.PRICE_PRERMT AS PRICE_CAD, (F.PRICE_PRERMT/ M.FXUSDCAD) AS PRICE_USD \n         FROM FARM_PRICES, MONTHLY_FX \n         WHERE F.DATE=M.DATE AND F.CORP_TYPE='Canola' AND GEO='Saskatchewan' ORDER BY F.DATE DESC LIMIT 6'"

## Author(s)

<h4> Jeff Grossman </h4>

## Contributor(s)

<h4> Rav Ahuja </h4>

## Change log

| Date       | Version | Changed by    | Change Description                                                                                          |
| ---------- | ------- | ------------- | ----------------------------------------------------------------------------------------------------------- |
| 2021-04-01 | 0.7     | Jeff Grossman | Split Problem 1 solution cell into multiple cells, fixed minor bugs |
| 2021-03-12 | 0.6     | Jeff Grossman | Cleaned up content for production |
| 2021-03-11 | 0.5     | Jeff Grossman | Moved more advanced problems to optional honours module |
| 2021-03-10 | 0.4     | Jeff Grossman | Added introductory and intermediate level problems and removed some advanced problems |
| 2021-03-04 | 0.3     | Jeff Grossman | Moved some problems to a new practice lab as prep for this assignment
| 2021-03-04 | 0.2     | Jeff Grossman | Sorted problems roughly by level of difficulty and relegated more advanced ones to ungraded bonus problems  |
| 2021-02-20 | 0.1     | Jeff Grossman | Started content creation                                                                                    |


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
